In [ ]:
# clone keras-retinanet from github and install it
!git clone https://github.com/fizyr/keras-retinanet.git
!pip install --upgrade keras
%cd keras-retinanet/
!pip install .

In [ ]:
# build keras-retinanet using script setup.py
!python setup.py build_ext --inplace

In [ ]:
# install tensorflow-gpu
!pip install tensorflow-gpu

In [ ]:
# set up environment
import io
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.patches as patches
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_detections
from keras_retinanet.utils.colors import label_color

In [ ]:
# grab and check out training data
path = '/kaggle/input/global-wheat-detection/'
train_data = pd.read_csv(path + 'train.csv')
train_data.head()

In [ ]:
# split into train and test sets
df_train, df_test = train_test_split(train_data,test_size=0.2, random_state=2020)

In [ ]:
# re-format train and test data to match requirements of keras-retinanet: path/to/image.jpg,x1,y1,x2,y2,class_name
# current format: xmin, ymin, width, height
def formatKRN(data):
    dataset = {'img': [], 'x1':[], 'y1':[], 'x2':[], 'y2':[], 'class':[]}
    for index, row in data.iterrows():
        dat = row['bbox']
        dat = dat.replace('[', '')
        dat = dat.replace(']', '')
        dat = dat.split(',')
        dat = [int(float(ii)) for ii in dat]
        dataset['img'].append(path + 'train/' + row['image_id'] + '.jpg')
        dataset['x1'].append(dat[0])
        dataset['y1'].append(dat[1])
        dataset['x2'].append(dat[0] + dat[2])
        dataset['y2'].append(dat[1] + dat[3])
        dataset['class'].append('whead')
    
    return pd.DataFrame(dataset)

df_train = formatKRN(data=df_train)
df_test = formatKRN(data=df_test)

print(df_train.shape, df_test.shape)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# function for plotting image and bboxes
def draw_train_img_and_bboxes(df, i, col):
    imgid = df['img'][i]
    img = Image.open(imgid)
    bboxes = df[df.img == imgid][['x1', 'y1', 'x2', 'y2']]
    fig,ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(img)
    for index, row in bboxes.iterrows():    
        rect = patches.Rectangle(xy=(row['x1'], row['y1']), 
                                 width=row['x2']-row['x1'], 
                                 height=row['y2']-row['y1'], 
                                 linewidth=1, edgecolor=col, facecolor='none')
        ax.add_patch(rect)
        plt.title(imgid)

def draw_test_img_and_bboxes(df, i, predboxes, colobs, colpred, minscore):
    imgid = df['img'][i]
    img = Image.open(imgid)
    bboxes = df[df.img == imgid][['x1', 'y1', 'x2', 'y2']]
    fig,ax = plt.subplots(1, figsize=(10, 10))
    
    # draw observed bboxes on test image
    for index, row in bboxes.iterrows():    
        rect = patches.Rectangle(xy=(row['x1'], row['y1']), 
                                 width=row['x2']-row['x1'], 
                                 height=row['y2']-row['y1'], 
                                 linewidth=1, edgecolor=colobs, facecolor='none')
        ax.add_patch(rect)        
        
    # draw predicted bboxes on test image
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < minscore: # (scores are sorted)
            break
        b = box.astype(int)
        rect = patches.Rectangle(xy=(b[0], b[1]), width=b[2], height=b[3], linewidth=1, 
                                 edgecolor=colpred, facecolor='none')
        ax.add_patch(rect)
    
    ax.imshow(img)

In [ ]:
# plot a sample image with boxes
draw_train_img_and_bboxes(df=df_train, i=12041, col='yellow')

In [ ]:
# files required for training
df_train.to_csv('trainDatFile.csv', index=False, header=None)
with open('classFile.csv', 'w') as f:
    f.write('{},{}\n'.format('whead',0))

In [ ]:
!head trainDatFile.csv

In [ ]:
!head classFile.csv

In [ ]:
# dowload pre-trained model (weights) to use as starting point in training
pretrainedMod = './snapshots/_pretrained_model.h5'
pretrainedModUrl = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(pretrainedModUrl, pretrainedMod)
print('Downloaded pretrained model to ' + pretrainedMod)

In [ ]:
# train with script train.py from keras_retinanet github repository ****************
!keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {pretrainedMod} \
--batch-size 4 \
--steps 200 \
--epochs 10 \
csv trainDatFile.csv classFile.csv

In [ ]:
# load trained model and convert to inference model (because we freezed backbone layer in training)
model = models.load_model('snapshots/resnet50_csv_10.h5', backbone_name='resnet50')
model = models.convert_model(model)

In [ ]:
# make prediction on single image (i=index in df_test) and plot together with observed bboxes
i = 909
image = read_image_bgr(df_test['img'][i])
image = preprocess_image(image)
image, scale = resize_image(image)
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
boxes /= scale # bring predicted boxes back to the images' original scale
draw_test_img_and_bboxes(df_test, 
                         i=i, 
                         predboxes=boxes, 
                         colobs='red', colpred='blue', 
                         minscore=0.65)